In [85]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.metrics import root_mean_squared_error
import plotly.express as px
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
start = datetime(2021, 1, 1)
end = datetime(2023, 12, 30)

bpac3 = yf.Ticker("BPAC3.SA")
#df_bpac3 = bpac3.history(start=start,end=end)
df_bpac3 = bpac3.history(start=start-relativedelta(days=10),end=end)

total_rows = len(df_bpac3)
legs = 21
qtd_features = 3
qtd_goal = 1
df_bpac3[['five_days_before']] = df_bpac3[['Close']].shift(5)

df_bpac3 = df_bpac3.loc[df_bpac3.index > '2021-01-01']

df_train = df_bpac3.iloc[:total_rows - legs]
df_test = df_bpac3.iloc[total_rows - legs:]

training_set_feature = df_train[['High', 'Low', 'five_days_before', 'Close']].values
testing_set_feature = df_test[['High', 'Low', 'five_days_before', 'Close']].values



In [86]:

# Responsável por fazer a divisão de elementos de eixo x(High e Low) e y(Close)
def split_sequence(sequence, n_steps, X_in, y_in):
	X, y = X_in, y_in
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return X, y

In [ ]:
X, y = list(), list()

# Pega os valores de High e Low e coloca em X e Close em y para todos os elementos de treinamento
for index in range(len(training_set_feature)):
    X, y = split_sequence(training_set_feature[index], qtd_features, X, y)
X, y = np.array(X), np.array(y)

#Cria o modelo
rnn = Sequential()

rnn.add(LSTM(units=50, return_sequences=True, input_shape=(qtd_features, qtd_goal)))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.5))

rnn.add(Dense(units=1))
rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

rnn.fit(X, y, epochs = 100, batch_size = 32)



In [ ]:
# Testa o elemento com os valores de High e Low para receber um valor de Close
qtd_features = 3 # Pois são 2 elementos high e low
qtd_goal = 1 # Pois quer predizer apenas 1, o close
y_real_test = []
y_predicted_test = []
for value in testing_set_feature:
    high = value[0]
    low = value[1]
    five_days_before = value[2]
    close_real = value[3]
    x_input = np.array([high, low, five_days_before])
    x_input = x_input.reshape((qtd_goal, qtd_features))
    yhat = rnn.predict(x_input, verbose=0)
    
    y_real_test.append(close_real)
    y_predicted_test.append(yhat[0][0])

root_mean_squared_error(y_real_test, y_predicted_test)



In [ ]:
print(y_real_test)
print("---")
print(y_predicted_test)